# OMMX Adapterで最適化問題を解く

OMMXでは、OMMXを既存の数理最適化ツールと相互連携するためのソフトウェアであるOMMX Adapterを提供します。OMMX Adapterを使うことで、OMMXが規定するスキーマで表現された最適化問題を様々な数理最適化ツールで解くことができます。

ここでは、0-1ナップサック問題をOMMX PySCIPOpt Adapterで解く方法を紹介します。


## 必要なライブラリのインポート

ここでは、OMMX Python SDKとOMMX PySCIPOpt Adapterを使います。これらのライブラリは、以下のコマンドでインストールできます： 

```
pip install ommx ommx_pyscipopt_adapter
```

## 最適化を解く2つのステップ

```{figure} ./assets/solve_with_ommx_adapter_01.png
:alt: 0-1ナップサック問題をOMMX PySCIPOpt Adapterで解くフロー

0-1ナップサック問題をOMMX PySCIPOpt Adapterで解くフロー。
```

0-1ナップサック問題をOMMX PySCIPOpt Adapterで解くためのステップは、次の2つあります：

1. 0-1ナップサック問題のインスタンスを用意する
2. OMMX Adapterを使って最適化する

ステップ1.では、OMMX MessageのInstanceスキーマで定義された `ommx.v1.Instance` オブジェクトを作成します。このオブジェクトを作成する方法は複数ありますが、ここではOMMX Python SDKを使用して直接記述する方法を採用します。

```{tip}
`ommx.v1.Instance` オブジェクトを用意する方法は4つあります：

1. OMMX Python SDKを使って `ommx.v1.Instance` を直接記述する
2. OMMX Adapterを使って数理最適化ツールで記述したインスタンスを `ommx.v1.Instance` に変換する
3. JijModelingを使って `ommx.v1.Instance` を出力する
4. OMMX Python SDKを使ってMPSファイルを `ommx.v1.Instance` に変換する
```

ステップ2.では、 `ommx.v1.Instance` をOMMX PySCIPOpt Adapterに入力して最適化計算を実行します。計算結果は、OMMX MessageのSolutionスキーマで定義された `ommx.v1.Solution` オブジェクトとして取得できます。

### ステップ1: 0-1ナップサック問題のインスタンスを用意する

0-1ナップサック問題は以下のように定式化されます：

$$
\begin{align*}
\mathrm{maximize} \quad & \sum_{i=0}^{N-1} v_i x_i \\
\mathrm{s.t.} \quad & \sum_{i=0}^{n-1} w_i x_i - W \leq 0, \\
& x_{i} \in \{ 0, 1\} 
\end{align*}
$$

この数理モデルのパラメーターに、以下のデータを設定します：

In [ ]:
# 0-1ナップサック問題のデータ
v = [10, 13, 18, 31, 7, 15]   # 各アイテムの価値
w = [11, 15, 20, 35, 10, 33] # 各アイテムの重さ
W = 47  # ナップサックの耐荷重
N = len(v)  # アイテムの総数

この数理モデルとデータに基づいて、OMMX Python SDKを使用してインスタンスを記述します：

In [ ]:
from ommx.v1 import Instance, DecisionVariable, Constraint

# 決定変数を定義する
x = [
    # バイナリ変数 x_i を定義する
    DecisionVariable.binary(
        # 決定変数のIDを指定する
        id=i,
        # 決定変数の名前を指定する
        name="x",
        # 決定変数の添え字を指定する
        subscripts=[i],
    )
    # バイナリ変数を num_items 個だけ用意する
    for i in range(N)
]

# 目的関数を定義する
objective = sum(v[i] * x[i] for i in range(N))

# 制約条件を定義する
constraint = Constraint(
    # 制約条件の名前
    name = "重量制限",
    # 制約式の左辺を指定する
    function=sum(w[i] * x[i] for i in range(N)) - W,
    # 等式制約 (==0) or 不等式制約 (<=0) を指定する
    equality=Constraint.LESS_THAN_OR_EQUAL_TO_ZERO,
)

# インスタンスを作成する
instance = Instance.from_components(
    # インスタンスに含まれる全ての決定変数を登録する
    decision_variables=x,
    # 目的関数を登録する
    objective=objective,
    # 全ての制約条件を登録する
    constraints=[constraint],
    # 最大化問題であることを指定する
    sense=Instance.MAXIMIZE,
)

### ステップ2: OMMX Adapterを使って最適化する

ステップ1で用意した0-1ナップサック問題のインスタンスをOMMX PySCIP Adapterで最適化します。最適化は、`ommx-pyscipopt-adapter` パッケージの `instance_to_model` と `model_to_solution` という2つの関数を使用することで実行できます：

In [ ]:
from ommx_pyscipopt_adapter import instance_to_model, model_to_solution

# ommx.v1.InstanceオブジェクトからPySCIPOptのModelオブジェクトに変換する
model = instance_to_model(instance)
# PySCIPOptで最適化計算を実行する
model.optimize()
# 計算結果をommx.v1.Solutionオブジェクトに格納して書き出す
solution = model_to_solution(model, instance)

ここで得られた変数 `solution` には、SCIPによる最適化計算の結果がOMMX MessageのSolutionスキーマに基づいて格納されています。

## 結果を分析する

ステップ2で得られた計算結果から

- 最適解（アイテムの価値の合計が最も高くなるようなアイテムの選び方）
- 最適値（最も高いアイテムの価値の合計）
- 制約条件（重量制限に対するアイテムの重さの合計の余裕）

を取得・分析するためには、`ommx.v1.Solution` クラスに実装されているプロパティを使用します。

### 最適解の分析

`decision_variables` プロパティは、決定変数のID、種類、名前、最適解などの情報を含む `pandas.DataFrame` オブジェクトを返します：

In [ ]:
solution.decision_variables

この `pandas.DataFrame` オブジェクトを使うことで、例えば「アイテムをナップサックに入れるかどうか」という判断をまとめた表をpandasで簡単に作成できます：

In [ ]:
import pandas as pd

df = solution.decision_variables
pd.DataFrame.from_dict(
    {
        "アイテムの番号": df.index,
        "ナップサックに入れるか？": df["value"].apply(lambda x: "入れる" if x == 1.0 else "入れない"),
    }
)

この分析結果から、ナップサックの重量制限を満たしながらアイテムの価値の合計を最大化するためには、0番目と3番目のアイテムを選択すればよいことが分かります。

### 最適値の分析

`objective` プロパティには最適値が格納されています。今回のケースでは、0番目と3番目のアイテムの価値の合計値が格納されているはずです：

In [ ]:
import numpy as np
# 期待される値は0番目と3番目のアイテムの価値の合計値である
expected = v[0] + v[3]
assert np.isclose(solution.objective, expected)

### 制約条件の分析

`constraints` プロパティは、制約条件の等号不等号、左辺の値 (`"value"`)、名前などの情報を含む `pandas.DataFrame` オブジェクトを返します：

In [ ]:
solution.constraints

重量制約を表す制約式は

$$
\begin{align*}
\sum_{i=0}^{n-1} w_i x_i - W \leq 0
\end{align*}
$$

であり、0番目と3番目のアイテムの重量の合計は

In [ ]:
assert w[0] + w[3] == 46

となります。$W$は `47` であることから、制約式の左辺の値は `-1` となるはずです。実際、 `solution.constraints` の列 `"value"` の値が `-1.0` となっているが確認できます。

このことから、最適解におけるアイテムの選び方では、重量制限に対して `1` だけ余裕があることが分かります。